# Applied Data Science Coursework
Name: Faris Ansara

In [3]:
import pandas as pd
import os
import warnings
import numpy as np
import math as m
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [4]:
# Supress warning messages
warnings.filterwarnings("ignore")

## 1. Functions to merge the csv files into Data frames

The first two function is to format the chosen additional data columns chosen for the analysis to be consistent

In [5]:
def format_additional_data_columns(df, additional_data_columns):
    """
    This functions formats the eco-friendly label columns in order to set false value when not any included in particular csv files that 
    include columns titled with any of the eco-frindly column titles listed in the list "eco_friendly_labels".

    Parameter:
    -df: Dataframe containing the data columns
    -eco_friendly_labels: list containg string titles of data columns containg whther the vehicles passing are eco-friendly

    Returns:
    - A dataframe with the formated eco-friendly column.
    """
    
    for col in df.columns:
        if col in additional_data_columns:
            #fill Nan values as False
            df[col].fillna(False, inplace=True)
    return df

In [6]:
def format_key_columns(df):
    """
    This fucntion formats the mandotory columns date, time, occupancy, direction and type. 

    Parameters:
    -df: Dataframe containing the data columns

    Returns:
    A dataframe where the date and time columns are formated to %d-%m-%Y and %I.%M %p respectively, occupancy set as string, and direction and type are set to lower case strings.
    """
    df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%d-%m')
    df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.strftime('%I.%M %p').str.lower().replace('pm', ' pm').replace('am', ' am')
    df['occupancy'] = df['occupancy'].astype(str)
    df['direction'] = df['direction'].astype(str).str.lower()
    df['type'] = df['type'].astype(str).str.lower()
    
    return df

This function is to concatenate the csv files into one Dataframe

In [7]:
def combine_csv_files(combined_data, file_path, additional_data_columns, key_columns):
    """
    This function concatenates csv files into a given dataframe according to the key columns in the csv files.

    Parameters:
    - combined_data: the initial dataframe to start with
    - file_path: file path of the folder that containes the csv files
    - additional_data_columns: a list data columns titles chosen to be your additional data
    - key_columns: a list of the titles of the key data columns

    Returns:
    - The combined dataframe with all the data in it.
    """
    for file_name in os.listdir(file_path):
        try:
            full_file_path = os.path.join(file_path, file_name)
            df = pd.read_csv(full_file_path, encoding='utf-8')
            #set column titles as lower case strings
            df.columns = df.columns.str.lower()

            # if not all the key columns are within the dataframe, skip it and move on to the next one
            if not all(col in df.columns for col in key_columns):
                continue

            # get the list of common columns that are within all the dataframes as these can be other columns not just the key columns
            common_columns = list(set(df.columns) & set(combined_data.columns))
            # format the data in the key columns
            df = format_key_columns(df)
            # format the additinial data columns chosen, these are eco-friendly and electric columns for 2023 data and just electric for 2022 data
            df = format_additional_data_columns(df, additional_data_columns)

            # check if the combined dataframe is empty it is set to equal the first dataframe
            if combined_data.empty:
                combined_data = df
            # else we concatanate the combined dataframe with the given dataframe
            else:
                combined_data = pd.concat([combined_data, df], axis=0, ignore_index=True, join='outer', keys=common_columns)
                
        except UnicodeDecodeError as e:
            # find the files causing proble,ms and delete them if necessary
            print(f"Error reading {file_name}: UnicodeDecodeError - {e}")

    return combined_data


The function below is used later to format the vehicle type 

In [8]:
def map_vehicle_type(x):
    """
    Function used to format vehicle names to a given format

    Paramters:
    - x: vehicle type

    Returns:
    - Formated vehicle type or None
    """
    # check if the value of x is not none, then we find the value inside the column and link it to the given key specified in the dectionary, set that key as the new value
    if x is not None:
        return next((key for key, values in type_ant.items() if x.lower() in values), None)
    return None

In [9]:
def format_occupancy(df):
    """
    Function used to format the occupancy in vehicles other than busses that are written in percentages to integer values

    Parameters:
    - df: the dataframe containg the values

    Returns:
    - The formated data frame
    """
    for index, col in df.iterrows():
        vehicle_type = col['type']
        occupancy = col['occupancy']
        
        if vehicle_type != 'bus' and "%" in str(occupancy):
            # Remove '%' sign and round up
            # Assuming 5-seater
            if vehicle_type in ['car', 'taxi', 'van']:
                occupancy = m.ceil(int(occupancy.rstrip("%")) / 20)
            # Assuming 1 rider
            elif vehicle_type in ['bicycle', 'motorcycle', 'scooter']:
                occupancy = 1
            # Assuming 2 seater
            elif vehicle_type in ['lorry', 'truck']:
                occupancy = m.ceil(int(occupancy.rstrip("%")) / 50)

           
        # Update the DataFrame with the modified occupancy valu
        if '%' not in str(occupancy):
            occupancy = pd.to_numeric(occupancy)
        df.at[index, 'occupancy'] = occupancy

    return df

In [10]:
def change_bus_occupancy_from_perc_to_int(df):  
    """
    Function used to format the occupancy of busses from percentages to integer values

    Parameters:
    - df: the dataframe containg the values

    Returns:
    - The formated data frame
    """      
    for index, row in df.iterrows():
        occupancy = row['occupancy']
        vehicle_type = row['type']
        if "%" in str(occupancy):
            occupancy = int(occupancy.rstrip('%'))
        if vehicle_type == 'bus':
            # assuming all buses are double deckers and fit 80 people at a full state
            # calculate the amount of people on the bus and round up
            occupancy = m.ceil((int(occupancy)/100) * 80)
            # Update the DataFrame with the modified occupancy value
            df.at[index, 'occupancy'] = occupancy
        
    return df

## 2. Combine and Clean 2023 Files

In [12]:
combined_data = pd.DataFrame()
folder_path = 'extracted_files_2023'
# additional data labels
eco_friendly_and_elec_labels = ['is_env_friendly', 'eco-friendly', 'low emissions compliant', 'green vehicle', 'is_env_fr', 'ecofriendly', 'ev' , 'is electric', 'ev', 'electric', 'zero emissions', 'zero emission' ]
key_columns = ['date', 'time', 'type', 'direction', 'occupancy']
combined_data_2023 = combine_csv_files(combined_data, folder_path ,eco_friendly_and_elec_labels ,key_columns)

Error reading 22792751.csv: UnicodeDecodeError - 'utf-8' codec can't decode byte 0xa0 in position 20: invalid start byte


Create eco-frindly column by concatinating columns that include eco-friendly labels

In [13]:
eco_friendly_labels = ['is_env_friendly', 'eco-friendly', 'low emissions compliant', 'green vehicle', 'is_env_fr', 'ecofriendly']
# Concatenate all columns with eco-friendly labels into a column named eco, 
# format values in ['yes', 'y', '1'] as True and the rest to False or None depending if they are not Null
combined_data_2023['eco'] = combined_data_2023.apply(
    lambda row: any(str(row[val]).lower() in ['yes', 'y', '1'] for val in eco_friendly_labels) if any(pd.notna(row[val]) for val in eco_friendly_labels) else None,
    axis=1
)

In [14]:
elec_labels = ['is electric', 'ev', 'electric', 'zero emissions', 'zero emission']
# Concatenate all columns with elec labels into a column named elec
# format values in ['yes', 'y', 'ev'] as True and the rest to False or None depending if they are not Null
combined_data_2023['elec'] = combined_data_2023.apply(
    lambda row: any(str(row[val]).lower() in ['yes', 'y', 'ev'] for val in elec_labels) if any(pd.notna(row[val]) for val in elec_labels) else None,
    axis=1
)

In [15]:
# Find all the values that make up the type column so that we can clean it up
# This is used in the next cell
vals = []
for val in combined_data_2023['type']:
    if val not in vals:
        vals.append(val)
print(vals)

['car', 'bus', 'taxi', 'motorbike', 'van', 'bicycle', 'lorry', 'bus ', 'bu', 'c', 'v', 'l', 's', 't', 'scooter', 'mini truck', 'bike', 'nan', 'motorcycle', 'other', 'truck', 'authorised vehicle', 'a.v', 'm', 'bi', 'motor', 'moterbike', 'u2', 'bicycle ', '5%', '1', '4', '1%', '3', '3%', '2', 'double decker bus', 'single decker bus']


In [16]:
# Created a dictionary with all the types of vehicles and the acronyms used for them that made sense usigng the list from before 
type_ant = {
    'car': ['car', 'c'],
    'bus': ['bus', 'bu', 'double decker bus','single decker bus'],
    'taxi': ['taxi'],
    'motorcycle': ['motorbike', 'motor', 'moterbike', 'motorcycle', 'm'],
    'van': ['van', 'v'],
    'bicycle': ['bicycle', 'bike', 'bicycle '],
    'lorry': ['lorry', 'l'],
    'scooter': ['scooter', 's'],
    'truck': ['truck', 'mini truck'],
}

# Clean type column to have consistant naming
combined_data_2023['type'] = combined_data_2023['type'].apply(map_vehicle_type)
# Drop rows where 'vehicle_type' is not found in any list
combined_data_2023.replace(['None', 'nan', ''], pd.NA, inplace=True)
combined_data_2023 = combined_data_2023.dropna(subset=['type', 'occupancy'])
combined_data_2023 = format_occupancy(combined_data_2023)

In [17]:
columns_to_keep = ['date', 'time', 'type', 'occupancy', 'direction', 'eco', 'elec']
combined_data_2023 = combined_data_2023[columns_to_keep].dropna(subset = key_columns)
combined_data_2023.to_csv('combined_data_2023.csv', index=False)

## 3. Combine and Clean 2022 Files

In [18]:
combined_data = pd.DataFrame()
folder_path = 'extracted_files_2022'
elec_labels = ['electric or not', 'electric vehicle', 'cartype', 'electric / non-electric vehicle', 'electric', 'electric_vehicle', 'fuel']
key_columns = ['date', 'time', 'type', 'direction', 'occupancy']
combined_data_2022 = combine_csv_files(combined_data, folder_path ,elec_labels, key_columns)

In [19]:
# Concatenate all columns with elec labels into a column named elec
# format values in ['yes', 'electric', '+'] as True and the rest to False or None depending if they are not Null
combined_data_2022['elec'] = combined_data_2022.apply(
    lambda row: any(str(row[val]).lower() in ['yes', 'electric', '+'] for val in elec_labels) if any(pd.notna(row[val]) for val in elec_labels) else None,
    axis=1
)

In [20]:
type_ant = {
    'car': ['car', 'c'],
    'bus': ['bus', 'bu', 'double decker bus', 'single decker bus'],
    'taxi': ['taxi'],
    'motorcycle': ['motorbike', 'motor', 'moterbike', 'motorcycle', 'm'],
    'van': ['van', 'v'],
    'bicycle': ['bicycle', 'bike', 'bicycle '],
    'lorry': ['lorry', 'l'],
    'scooter': ['scooter', 's'],
    'truck': ['truck', 'mini truck'],
}

# Clean type column to have consistant naming
combined_data_2022['type'] = combined_data_2022['type'].apply(map_vehicle_type)
# Drop rows where 'vehicle_type' is not found in any list
combined_data_2022.replace(['None', 'nan', 'medium', 'empty','Empty', 'Half-Full'], pd.NA, inplace=True)
combined_data_2022 = combined_data_2022.dropna(subset=['type', 'occupancy'])
combined_data_2022 = format_occupancy(combined_data_2022)

In [21]:
columns_to_keep = ['date', 'time', 'type', 'occupancy', 'direction', 'elec']
combined_data_2022 = combined_data_2022[columns_to_keep].dropna(subset = key_columns)
combined_data_2022.to_csv('combined_data_2022.csv', index=False)

## 4 Data Plotting

### 4.1 Plotting eco-friendly vs non-ecofriendly vehicles

In [24]:
# Drop Nan values in the "eco" column to get only True and Falase values
eco_friedly_and_not = combined_data_2023.reset_index().dropna(subset = ['eco'])
# Group by "eco" to do analysis
groupby_eco = eco_friedly_and_not.groupby('eco')

In [25]:
# Plot pie chart of Eco vs Non-Eco using Plotly
labels = ['Non-Ecofriendly', 'Eco-Friendly']
values = groupby_eco['eco'].count()

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3, marker=dict(colors=['#1076BF', '#36A465']))])

fig.update_layout(title_text='Eco-friendly vs Non-Ecofriendly vehicles entering and leaving the University 2023',
                  showlegend=True,
                  paper_bgcolor='#E5ECF6')

# save fig as pdf
fig.write_image("figures\eco.pdf")

fig.show()

### 4.2 Plotting green house gases

In [26]:
# Plot percanges of Contribution of green houses gases UK
labels = ['Carbon Dioxide', 'Other green-house Gases']
values = [80, 20]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0.2, 0], marker=dict(colors=['black', 'gray']))])

fig.update_layout(title_text='Carbon Dioxide Contribution of All Green-House-Gas Emission in the Uk',
                  showlegend=True,
                  paper_bgcolor='#E5ECF6')

fig.write_image("figures\green_house_gases.pdf")

fig.show()

### 4.3 Plotting GHG (Green House Gas) emmisions by sector, 2021

In [27]:
# Plot GHG emmisions by sector in the UK 2021
labels = ['Transport', 'Energy Supply', 'Buisness','Residential', 'Agriculture', 'Other']
values = [26, 20, 18, 16, 11, 9]

fig = go.Figure(go.Bar(
    x=labels,
    y=values,
    marker=dict(color=['black','blue', 'skyblue' , 'lightgreen', 'gray', 'lightgray']))
)

fig.update_layout(title_text='Greenhouse Gas Emissions By Sector, 2021',
        xaxis_title='Sector',
        yaxis_title='Percentage contricution (%)',
        showlegend=False,
        paper_bgcolor='#E5ECF6')

fig.write_image("figures\ghg_emmisions_by_sec.pdf")

fig.show()

### 4.4 Plotting Elec diff 2022 2023

In [28]:
# drop na values in elec columnn in 2023 data
elec_and_not_2023 = combined_data_2023.dropna(subset = ['elec'])
# create new data frame with only true values for analysis
elec_and_not_2023_only_elec = elec_and_not_2023[elec_and_not_2023['elec']].reset_index()

# drop na values in elec columnn in 2022 data
elec_and_not_2022 = combined_data_2022.dropna(subset = ['elec'])
# create new data frame with only true values for analysis
elec_and_not_2022_only_elec = elec_and_not_2022[elec_and_not_2022['elec']].reset_index()

# group both data frames for 2022 and 2023 by 'elec' column
groupby_elec_2023 = elec_and_not_2023_only_elec.groupby('elec')
groupby_elec_2022 = elec_and_not_2022_only_elec.groupby('elec')

# calculate perc by diving number of true values by total number of data points then multply by 100
elec_2023_perc = (int(groupby_elec_2023['elec'].count()) / len(elec_and_not_2023)) * 100
elec_2022_perc = (int(groupby_elec_2022['elec'].count()) / len(elec_and_not_2022)) * 100

In [29]:
# plot percentage of electric cars in 2022 and 2023 using plotly
labels = ['2022', '2023']
values = [elec_2022_perc, elec_2023_perc]
text_on_bars = [str(round(perc, 2)) + '%' for perc in values]

fig = go.Figure(go.Bar(
    x=labels,
    y=values,
    text=text_on_bars,
    marker = dict(color=['skyblue'])
))

fig.update_layout(title_text='Percentage of Electric Entering and Leaving the University in 2022 vs 2023',
        xaxis_title='Year',
        yaxis_title='Percentage of Electric Cars (%)',
        showlegend=False,
        paper_bgcolor='#E5ECF6')

fig.write_image("figures\elec_2022_2023.pdf")

fig.show()

### 4.5 Plotting occupancy in busses vs other means of transportation difference between 2022 and 2023

In [30]:
# create data frames with just type and occupancy to not edit the original dataframe
type_and_occupancy_2023 = combined_data_2023[['type', 'occupancy']]
type_and_occupancy_2022 = combined_data_2022[['type', 'occupancy']]

# change the bus occupancies from percentages to integers to make it possible to compare
# with other means of transportation
type_and_occupancy_2023 = change_bus_occupancy_from_perc_to_int(type_and_occupancy_2023)
type_and_occupancy_2022 = change_bus_occupancy_from_perc_to_int(type_and_occupancy_2022)

# group the dataframes by type and get the sum of the occupancies for each vehicle type
# we can use this latter to create pie charts
sum_of_occupancies_2023 = type_and_occupancy_2023.groupby('type').sum().reset_index()
sum_of_occupancies_2022 = type_and_occupancy_2022.groupby('type').sum().reset_index()

In [31]:
# create pie chart for 2022 data
fig2 = go.Figure(data=[go.Pie(labels=sum_of_occupancies_2022['type'],
                              values=sum_of_occupancies_2022['occupancy'],
                              pull=[0.05 if t == 'bus' else 0 for t in sum_of_occupancies_2022['type']])])


# create pie chart for 2023 data
fig1 = go.Figure(data=[go.Pie(labels=sum_of_occupancies_2023['type'],
                              values=sum_of_occupancies_2023['occupancy'],
                              pull=[0.05 if t == 'bus' else 0 for t in sum_of_occupancies_2023['type']])])

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=['2022', '2023'], specs=[[{'type': 'pie'}, {'type': 'pie'}]])

# Add pie charts to subplots
fig.add_trace(fig1.data[0], row=1, col=2)
fig.add_trace(fig2.data[0], row=1, col=1)

# Update layout for the main figure
fig.update_layout(title_text='Distribution of Transportation Modes: Percentage of Individuals Utilizing Each Method',
                  showlegend=True,
                  paper_bgcolor='#E5ECF6')

# save fig as pdf (the titles were edited on canva to fit the page)
fig.write_image("figures\ bus_usage.pdf")

fig.show()

### 4.6 plotting CO2 emmisions of avrg bus vs avrg car

In [33]:
# Plot Co2 emmision avrgs for bus vs car using plotly
labels = ['Car', 'Bus']
values = [170, 96]

fig = go.Figure(go.Bar(
    x=labels,
    y=values,
    marker = dict(color=['gray', 'green']
)))

fig.update_layout(title_text='Average Carbon Dioxide Emissions Per Passenger Per Kilometer on Buses Vs Cars',
        xaxis_title='Vehicle',
        yaxis_title='Carbon Dioxide Emissions Per Passenger Per Kilometer (g/(km Passenger)',
        showlegend=False,
        paper_bgcolor='#E5ECF6')
        
fig.write_image("figures\co2_emmisions_per_km_passenger.pdf")

fig.show()


### 4.7 Plotting difference in CO2 emmisions due to buses and cars 2022 2023

In [35]:
# calculate average number of people going in and out of uni a day between 8-11am
passengers_2022 = np.sum(sum_of_occupancies_2022['occupancy'])
passengers_2023 = np.sum(sum_of_occupancies_2023['occupancy'])
avg_passengers_per_day = round((passengers_2022 + passengers_2023)/10)

In [36]:
# take the median of emmisions of cars and busses
car_emissions = 170
bus_emissions = 92

In [37]:
# calculate ratio of busses to cars in 2022 and 2023
ratio_buses_2022 = float(sum_of_occupancies_2022[sum_of_occupancies_2022['type'] == 'bus']['occupancy'] / sum_of_occupancies_2022["occupancy"].sum())
ratio_cars_2022 = float(sum_of_occupancies_2022[sum_of_occupancies_2022['type'] == 'car']['occupancy'] / sum_of_occupancies_2022["occupancy"].sum())

ratio_cars_2023 = float(sum_of_occupancies_2023[sum_of_occupancies_2023['type'] == 'car']['occupancy'] / sum_of_occupancies_2023["occupancy"].sum())
ratio_buses_2023 = float(sum_of_occupancies_2023[sum_of_occupancies_2023['type'] == 'bus']['occupancy'] / sum_of_occupancies_2023["occupancy"].sum())

In [38]:
# calculate Co2 emissions per day in 2023 vs 2022
Co2_emmisions_2023 = (ratio_buses_2023*bus_emissions + ratio_cars_2023*car_emissions)*avg_passengers_per_day
Co2_emmisions_2022 = (ratio_buses_2022*bus_emissions + ratio_cars_2022*car_emissions)*avg_passengers_per_day

# calculate diff in emissions between 2023 and 2022
diff_emissions_22_23 = abs(Co2_emmisions_2023 - Co2_emmisions_2022)

In [40]:
# plot bar graph of difference in CO2 emmisions per day 2022 vs 2023
fig = go.Figure(go.Bar(
    x=['2022', '2023'],
    y=[Co2_emmisions_2022, Co2_emmisions_2023],
    marker=dict(color=['#36A465', '#1076BF'], coloraxis="coloraxis")
))

fig.update_layout(title_text='CO2 Emissions Comparison between 2022 and 2023.'.format(diff_emissions_22_23),
        xaxis_title='Year',
        yaxis_title='Grams of CO2 per kilometer per day (g/km per day)',
        showlegend=False,
        paper_bgcolor='#E5ECF6')

fig.write_image("figures\ Co2_emmision_diff.pdf")

fig.show()